In [1]:
know = {
    "intents" : {
        "saudacao" : [
            "Oi", "Olá"
        ],
        "despedida" : [
            "tchau", "Até mais!"
        ],
        "saldo" : [
            "Qual é meu saldo?", "Quanto tenho na conta?", "Saldo da conta corrente", "Quantas pilas eu tenho?", "Qual é o saldo da minha conta?"
        ],
        "extrato" : [
            "Extrato da conta", "Extrato da conta corrente"
        ],
        "recarregar-celular" : [
            "Quero recarregar meu celular!"
        ],
        "pagar-receber" : [
            "link pra receber"
        ],
        "transferir" : [
            "Transferir dinheiro"
        ]
    }
}
print(know);

{'intents': {'transferir': ['Transferir dinheiro'], 'saldo': ['Qual é meu saldo?', 'Quanto tenho na conta?', 'Saldo da conta corrente', 'Quantas pilas eu tenho?', 'Qual é o saldo da minha conta?'], 'pagar-receber': ['link pra receber'], 'extrato': ['Extrato da conta', 'Extrato da conta corrente'], 'despedida': ['tchau', 'Até mais!'], 'saudacao': ['Oi', 'Olá'], 'recarregar-celular': ['Quero recarregar meu celular!']}}


In [2]:
dialog_welcome = {
    "condition": "#saudacao",
    "output": {
        "text": "Olá, eu sou o David!"
    }
}

dialog_nodes = [dialog_welcome,{
    "condition": "#despedida",
    "output": {
        "text": "Muito obrigado pela atenção! Precisando estou por aqui!"
    } 
}, {
    "condition": "#saldo",
    "output": {
        "text": "Seu saldo é R$ #.###,##!"
    } 
},{
    "condition": "#extrato",
    "output": {
        "text": "Segue seu extrato!!!"
    } 
},{
    "condition": "#recarregar-celular",
    "output": {
        "text": "Celular recarregado!"
    } 
},{
    "condition": "#pagar-receber",
    "output": {
        "text": "Segue o link!"
    } 
},{
    "condition": "#transferir",
    "output": {
        "text": "Dinheiro transferido!"
    } 
}]

dialog_anythinelse = { 
    "condition" : "true",
    "output": {
        "text": "Desculpe, não entendi! Pode falar de outra forma?"
    }
}

dialog_nodes.append(dialog_anythinelse)

print(dialog_nodes);

[{'condition': '#saudacao', 'output': {'text': 'Olá, eu sou o David!'}}, {'condition': '#despedida', 'output': {'text': 'Muito obrigado pela atenção! Precisando estou por aqui!'}}, {'condition': '#saldo', 'output': {'text': 'Seu saldo é R$ #.###,##!'}}, {'condition': '#extrato', 'output': {'text': 'Segue seu extrato!!!'}}, {'condition': '#recarregar-celular', 'output': {'text': 'Celular recarregado!'}}, {'condition': '#pagar-receber', 'output': {'text': 'Segue o link!'}}, {'condition': '#transferir', 'output': {'text': 'Dinheiro transferido!'}}, {'condition': 'true', 'output': {'text': 'Desculpe, não entendi! Pode falar de outra forma?'}}]


In [3]:
stopwords = set(line.strip() for line in open('stopwords.txt', 'r', encoding='utf-8'))

In [4]:
import re

NON_CONTENT = r"[^\w\d\s]"
def tokenize(sentence):
    #remove non content
    sentence = re.sub(NON_CONTENT, "", sentence)
    #lower
    sentence = sentence.lower();
    
    #split
    tokens = sentence.split(" ");
        
    for sw in stopwords:
        try:
            tokens.remove(sw); 
        except ValueError:
            pass
    
    return tokens;

print(tokenize("Qual é o meu saldo?"))

['saldo']


In [5]:
intent_model = {}

for intent, samples in know["intents"].items():
    intent_model[intent] = {}
    for sample in samples:
        intent_model[intent][sample] = {
            "total" : 0,
            "tokens" : {}
        }
        for t in tokenize(sample):
            intent_model[intent][sample]["total"] += 1;
            if t in intent_model[intent][sample]["tokens"]:
                intent_model[intent][sample]["tokens"][t] += 1;
            else:
                intent_model[intent][sample]["tokens"][t] = 1;
    
print("intent_model:", intent_model);    

intent_model: {'transferir': {'Transferir dinheiro': {'total': 2, 'tokens': {'transferir': 1, 'dinheiro': 1}}}, 'saldo': {'Quanto tenho na conta?': {'total': 2, 'tokens': {'tenho': 1, 'conta': 1}}, 'Saldo da conta corrente': {'total': 3, 'tokens': {'conta': 1, 'saldo': 1, 'corrente': 1}}, 'Qual é meu saldo?': {'total': 1, 'tokens': {'saldo': 1}}, 'Quantas pilas eu tenho?': {'total': 2, 'tokens': {'pilas': 1, 'tenho': 1}}, 'Qual é o saldo da minha conta?': {'total': 3, 'tokens': {'minha': 1, 'saldo': 1, 'conta': 1}}}, 'pagar-receber': {'link pra receber': {'total': 3, 'tokens': {'pra': 1, 'link': 1, 'receber': 1}}}, 'extrato': {'Extrato da conta corrente': {'total': 3, 'tokens': {'conta': 1, 'corrente': 1, 'extrato': 1}}, 'Extrato da conta': {'total': 2, 'tokens': {'conta': 1, 'extrato': 1}}}, 'despedida': {'Até mais!': {'total': 2, 'tokens': {'mais': 1, 'até': 1}}, 'tchau': {'total': 1, 'tokens': {'tchau': 1}}}, 'saudacao': {'Oi': {'total': 1, 'tokens': {'oi': 1}}, 'Olá': {'total': 1, 

In [6]:
def classify(input):
    tokens = tokenize(input);
    #print ("tokens", tokens);
    intents = {}
    for intent, samples in intent_model.items():
        intents[intent] = 0
        for s, smeta in samples.items():
            brutal_score = 0
            stokens = smeta["tokens"]
            for t in tokens:
                if t in stokens:
                    brutal_score += stokens[t]
            score = brutal_score / smeta["total"]
            if intents[intent] < score:
                intents[intent] = score
            
    intents = [{
        "intent": intent,
        "confidence": intents[intent]
    } for intent in sorted(intents, key=intents.__getitem__, reverse = True)]
    
    intents = list(filter(lambda i: i["confidence"] > 0, intents))
    
    return intents[:10];

print(classify("Qual meu saldo?"))

[{'intent': 'saldo', 'confidence': 1.0}]


In [7]:
def nlp(input):
    return [];

In [8]:
def evalCondition(condition, context, intent, entities):
    return condition == '#' + intent;

In [9]:
def dialog(input, context, intents, entities):
    if input == "":
        return dialog_welcome;
    
    if len(intents) > 0:
        intent = intents[0]["intent"];
        for dialog_node in dialog_nodes:
            if evalCondition(dialog_node["condition"], context, intent, entities):
                return dialog_node;
    
    return dialog_anythinelse;

In [10]:
def respond(input, context = {}):
    intents = classify(input)
    entities = nlp(input)
    #print("intents", intents)
    dialog_node = dialog(input, context, intents, entities)
    return {
        "context": context,
        "intents": intents,
        "entities": entities,
        "output": dialog_node["output"]
    }

In [11]:
def test_respond(input):
    resp = respond(input);
    #print("\n\nresp", resp);
    print("\ninput: ", input, " >> ", resp["output"]["text"]);

test_respond("");
test_respond("Oi");
test_respond("Qual meu saldo?");
test_respond("Quanto tenho na conta?");
test_respond("Extrato da conta");
test_respond("link pra receber");
test_respond("Horário das agências");
test_respond("recarregar celular");
test_respond("Então tchau");


input:    >>  Olá, eu sou o David!

input:  Oi  >>  Olá, eu sou o David!

input:  Qual meu saldo?  >>  Seu saldo é R$ #.###,##!

input:  Quanto tenho na conta?  >>  Seu saldo é R$ #.###,##!

input:  Extrato da conta  >>  Segue seu extrato!!!

input:  link pra receber  >>  Segue o link!

input:  Horário das agências  >>  Desculpe, não entendi! Pode falar de outra forma?

input:  recarregar celular  >>  Celular recarregado!

input:  Então tchau  >>  Muito obrigado pela atenção! Precisando estou por aqui!
